<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/CNN_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 27.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import pytorch_lightning as pl
#from pytorch_lightning.logging import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [3]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [4]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [6]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [7]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [8]:
train = pd.read_csv(path+"/train.csv")
validation = pd.read_csv(path+"/validation.csv")
test = pd.read_csv(path+"/boneage-test-dataset.csv")

In [9]:
train

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [10]:
import scipy.stats as stats

In [11]:
#train['id'] = train['id'].apply(lambda x: str(x)+'.png')
train['boneage zscore'] = stats.zscore(train['boneage'])
#train['boneage'] = train['boneage'].apply(lambda x: str(x))
train['boneage zscore'] = train['boneage zscore'].apply(lambda x: int(x))
train['male'] = train['male'].apply(lambda x: 0 if x else 1)

train = pd.DataFrame(train)
train.rename(columns={'male':'gender'},inplace=True)

In [12]:
train

,id,boneage,gender,boneage zscore
0,1377,180,1,1
1,1378,12,1,-2
2,1379,94,1,0
3,1380,120,0,0
4,1381,82,1,-1
...,...,...,...,...
10084,13259,94,1,0
10085,13261,94,1,0
10086,13274,94,1,0
10087,13286,94,1,0


In [13]:
#validation['id'] = validation['id'].apply(lambda x: str(x)+'.png')
validation['boneage zscore'] = stats.zscore(validation['boneage'])
#validation['boneage'] = validation['boneage'].apply(lambda x: str(x))
validation['boneage zscore'] = validation['boneage zscore'].apply(lambda x: int(x))
validation['male'] = validation['male'].apply(lambda x: 0 if x else 1)

validation = pd.DataFrame(validation)
validation.rename(columns={'male':'gender'},inplace=True)

In [14]:
validation

,id,boneage,gender,boneage zscore
0,10186,10,0,-2
1,10758,15,1,-2
2,10860,180,1,1
3,10877,180,1,1
4,11008,180,1,1
...,...,...,...,...
2517,15605,50,1,-1
2518,15606,113,1,0
2519,15608,55,1,-1
2520,15609,150,0,0


In [15]:
test

,id,boneage,male
0,4360,169,True
1,4361,170,True
2,4362,73,True
3,4363,153,True
4,4364,135,True
...,...,...,...
195,4555,133,False
196,4556,129,False
197,4557,168,False
198,4558,136,False


In [16]:
#test['id'] = test['id'].apply(lambda x: str(x)+'.png')
test['boneage zscore'] = stats.zscore(test['boneage'])
#test['boneage'] = test['boneage'].apply(lambda x: str(x))
test['boneage zscore']  = test['boneage zscore'].apply(lambda x: int(x))
test['male'] = test['male'].apply(lambda x: 0 if x else 1)

test = pd.DataFrame(test)
test.rename(columns={'male':'gender'},inplace=True)

In [17]:
test

,id,boneage,gender,boneage zscore
0,4360,169,0,0
1,4361,170,0,0
2,4362,73,0,-1
3,4363,153,0,0
4,4364,135,0,0
...,...,...,...,...
195,4555,133,1,0
196,4556,129,1,0
197,4557,168,1,0
198,4558,136,1,0


In [18]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, data, image_dir):
        self.image_dir = image_dir
        #self.pickle_file = pickle_file
        #self.frame = frame
        self.data = data
        self.table = pd.DataFrame(data)


    def __len__(self):
        return len(self.table)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        table = self.table.iloc[idx, 0:]

        y = table["boneage zscore"]

        image = Image.open(f"{self.image_dir}/{table['id']}.png")
        image = np.array(image)
        #image = image[..., :3]

        image = transforms.functional.to_tensor(image)
        image = transforms.functional.resize(image, size=[500,500])
        # image = transforms.Compose([transforms.Resize((500, 500)),
        #                        transforms.RandomAffine(
        #                            20, translate=(0.2, 0.2),
        #                            scale=(1, 1.2)),
        #                        transforms.RandomHorizontalFlip(),
        #                        transforms.ToTensor()])

        table = table[["gender"]]
        table = table.tolist()
        table = torch.FloatTensor(table)

        return image, table, y

In [19]:
np.array(Image.open(f"{train_img}{train.iloc[0,0:]['id']}.png"))[..., :3].shape

(2044, 3)

In [20]:
ImageDataset(train, image_dir=f"{train_img}")

In [21]:
class CNN(nn.Module):
    def __init__(self,transform_input = False, lr = 0.0001, num_workers = 0, batch_size = 20):
        super(CNN, self).__init__()
        
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size
        
        self.conv1 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # input height
                out_channels=16,            # n_filters
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (16, 28, 28)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(16, 32, 5, 1, 2),     # output shape (32, 14, 14)
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2),                # output shape (32, 7, 7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)   # fully connected layer, output 10 classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output, x    # return x for visualization
    
    def training_step(self, batch, batch_idx):
      image, table, y = batch

      criterion = torch.nn.L1Loss()
      y_pred = torch.flatten(self(image, table))
      y_pred = y_pred.double()

      loss = criterion(y_pred, y)

      tensorboard_logs = {"train_loss": loss}
      return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        image, table, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, table))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred, y)

        return {"val_loss": val_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"val_loss": avg_loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        image, table, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, table))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred, y)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}


In [22]:
    def setup(self, stage):

        image_datatrain = ImageDataset(train, image_dir=f"{train_img}")
        image_dataval = ImageDataset(validation, image_dir=f"{val_img}")
        image_datatest = ImageDataset(test, image_dir=f"{test_img}")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest

        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=(self.lr),)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

In [23]:
from fastai.vision.learner import Learner
from fastai.callback.schedule import lr_find
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import nn, optim, rand, sum as tsum, reshape, save
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint

In [24]:
if __name__ == '__main__':
    #logger = TensorBoardLogger("lightning_logs", name="image_only")
    #early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=5000, patience=7, verbose=True, mode='min')
    
    logger = pl.loggers.TensorBoardLogger("lightning_logs",version = 'version_0', name="my_model")
    checkpoint_callback = ModelCheckpoint(dirpath = "lightning_logs",verbose=True, mode='min',every_n_train_steps = None, every_n_epochs = None)
    model = CNN()
    trainer = Trainer(accelerator='gpu', devices=1, logger=logger,max_epochs=5, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,callbacks=[checkpoint_callback])
                    #callbacks=[early_stop_callback])
    #trainer.tune(model)

    trainer.fit(model)
    trainer.test(model)
    save(model.state_dict(), 'Location of our saved model')

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


TypeError: ignored